# Actividad Guiada 3

# Marco Antonio Arcidiacono Alepuz

Problema del agente viajero y parte extra

https://github.com/marcoantonio135/03MIAR-Algoritmos-de-Optimizacion

## Carga de librerias

In [10]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

## Carga de los datos del problema

In [11]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random                     #Para generar valores aleatorios

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos



gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? y


In [12]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())



In [13]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html

#dir(problem)

15

## Funcionas basicas


In [14]:

#Funcionas basicas
###############################################################################

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)




## BUSQUEDA ALEATORIA

In [15]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto

  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)

    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia


  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion


#Busqueda aleatoria con 5000 iteraciones
solucion = busqueda_aleatoria(problem, 10000)

Mejor solución: [0, 23, 24, 22, 40, 8, 17, 31, 32, 29, 26, 35, 16, 14, 36, 15, 13, 6, 30, 5, 1, 3, 18, 25, 12, 21, 20, 34, 33, 41, 10, 9, 11, 38, 37, 39, 27, 4, 19, 7, 28, 2]
Distancia     : 3687


## BUSQUEDA LOCAL

In [16]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))


nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))


Distancia Solucion Incial: 3687
Distancia Mejor Solucion Local: 3335


In [17]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina


sol = busqueda_local(problem )

En la iteracion  37 , la mejor solución encontrada es: [0, 18, 12, 11, 25, 10, 8, 9, 21, 23, 41, 29, 28, 27, 32, 30, 38, 22, 39, 24, 40, 26, 5, 14, 16, 15, 37, 7, 17, 36, 35, 31, 20, 33, 34, 19, 13, 6, 4, 2, 3, 1]
Distancia     : 1775


### Parte extra

In [26]:
def busqueda_local_multiarranque(problem):
  arranques = 10 #número de arranques
  mejores_soluciones = [[] for i in range(arranques)]
  solucion_referencia = [crear_solucion(Nodos) for i in range(arranques)]
  mejores_distancias = [distancia_total(solucion_referencia[i], problem) for i in range(arranques)]
  encontrado =  [False for i in range(arranques)]

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecinas = [genera_vecina(solucion_referencia[i]) for i in range(arranques)]
    distancias_vecinas = [distancia_total(vecinas[i], problem) for i in range(arranques)]
    acabar = True
    for i in range(arranques):
      if (encontrado[i]==False):
        acabar = False
        #vecinas[i] = genera_vecina(solucion_referencia[i])
        #print(vecinas)
        #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
        #distancias_vecinas = distancias_vecinas + distancia_total(vecinas[i], problem)

        #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
        if distancias_vecinas[i] < mejores_distancias[i]:
          #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
          mejores_soluciones[i] = vecinas[i]                   #Guarda la mejor solución encontrada
          mejores_distancias[i] = distancias_vecinas[i]

        else:
          encontrado[i] = True
          print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejores_soluciones[i], "para el arranque:" , i)
          print("Distancia     :" , mejores_distancias[i])
          #return mejor_solucion

        solucion_referencia[i] = vecinas[i]
    if (acabar == True):
      mejor_sol = mejores_soluciones[0]
      mejor_dist = mejores_distancias[0]
      for i in range(1, arranques):
        if mejores_distancias[i] < mejor_dist:
          mejor_sol = mejores_soluciones[i]
          mejor_dist = mejores_distancias[i]

      print("La mejor solución encontrada es:" , mejor_sol)
      print("Distancia     :" , mejor_dist)
      return mejor_sol





sol = busqueda_local_multiarranque(problem )

En la iteracion  28 , la mejor solución encontrada es: [0, 27, 2, 3, 4, 10, 25, 41, 23, 9, 22, 38, 29, 8, 11, 12, 18, 14, 16, 15, 36, 35, 33, 20, 31, 17, 37, 7, 1, 32, 34, 24, 40, 21, 39, 30, 28, 26, 5, 13, 19, 6] para el arranque: 5
Distancia     : 1843
En la iteracion  31 , la mejor solución encontrada es: [0, 41, 23, 21, 39, 31, 17, 36, 35, 20, 30, 29, 28, 27, 2, 8, 9, 40, 24, 22, 38, 33, 34, 32, 11, 12, 18, 26, 5, 14, 15, 16, 19, 13, 25, 10, 6, 37, 7, 1, 4, 3] para el arranque: 7
Distancia     : 2057
En la iteracion  34 , la mejor solución encontrada es: [0, 3, 27, 2, 8, 9, 22, 38, 34, 33, 20, 35, 36, 31, 17, 1, 12, 11, 25, 41, 23, 40, 24, 21, 39, 28, 4, 6, 26, 18, 10, 29, 30, 32, 7, 37, 15, 16, 14, 19, 13, 5] para el arranque: 0
Distancia     : 1676
En la iteracion  35 , la mejor solución encontrada es: [0, 32, 28, 2, 27, 3, 4, 6, 26, 5, 14, 19, 13, 18, 29, 24, 40, 21, 23, 41, 10, 1, 7, 31, 35, 36, 16, 15, 37, 17, 20, 33, 34, 12, 11, 25, 8, 9, 39, 22, 38, 30] para el arranque: 9
D